# Dynamic Quantum Chemistry Workflow 

In this tutorial, we discuss how _covalent_ can be used to construct and manage **dynamic** workflows that are common in quantum-chemical simulations. In this example, given a crystal structure, we seek to do the following:

1. Relax the atomic positions and cell volume for the structure (i.e. find the local minimum energy configuration).
2. Carve all possible surface slabs of the relaxed structure. The number of slabs is only determined at runtime.
3. For each generated slab, relax the atomic positions of each structure.

This will be done using the materials informatics library [Pymatgen](https://github.com/materialsproject/pymatgen) and a machine learned interatomic potential called [M3GNet](https://www.nature.com/articles/s43588-022-00349-3). Because the number of slabs is only known at runtime, this is a dynamic workflow and we must take advantage of the sublattice feature in Covalent to properly dispatch the workflow.

First, we need to install the necessary Python packages from the `requirements.txt` file. These are printed below:

In [ ]:
with open("requirements.txt", "r") as file:
    for line in file:
        print(line.rstrip())

Once that is done, we can import the necessary functions for this tutorial:

In [ ]:
from subprocess import Popen
import covalent as ct
from pymatgen.core import Structure
from pymatgen.core.surface import generate_all_slabs

We then will make sure the Covalent server is started:

In [ ]:
sts = Popen("covalent start", shell=True).wait()

Now we will define the individual compute tasks. The first `Electron` we will define is a `relax_structure` function, which takes in a Pymatgen `Structure` object as the input and runs the relaxation calculation using M3GNet. It returns an updated `Structure` object. This process can be thought of as the compute-heavy task.

In [ ]:
@ct.electron
def relax_structure(structure, relax_cell=True):
    return structure.relax(relax_cell=relax_cell)

Now we will define a separate `Electron` called `carve_slabs` that takes in a `Structure` object and carves all plausible surface slabs, which it returns as list of new `Structure` objects. This is a relatively quick task, but we still need to define it as an `Electron`.

In [ ]:
@ct.electron
def carve_slabs(structure, max_index=1, min_slab_size=10.0, min_vacuum_size=10.0):
    return generate_all_slabs(
        structure,
        max_index,
        min_slab_size,
        min_vacuum_size,
    )

Now, for the most subtle but important bit. We need to define a sublattice that will take in the list of slabs and relax each individual slab using the previously defined `relax_structure` `Electron`. The sublattice is crucial here because the number of slabs is only determined at runtime.

In [ ]:
@ct.electron
@ct.lattice
def relax_slabs(slabs):
    return [relax_structure(slab, relax_cell=False) for slab in slabs]

With these individual `Electron` objects defined, we can now define the overall workflow that stitches them together and forms a `Lattice`. As introduced at the start of this tutorial, there are three major sets of tasks, and those are reflected by the three functions defined above.

In [ ]:
@ct.lattice(executor="local")
def workflow(structure):
    relaxed_structure = relax_structure(structure)
    slabs = carve_slabs(relaxed_structure)
    relaxed_slabs = relax_slabs(slabs)
    return relaxed_slabs

Now we can create an example `Structure` as the input for our workflow.

In [ ]:
structure = Structure(
    lattice=[[0, 2.13, 2.13], [2.13, 0, 2.13], [2.13, 2.13, 0]],
    species=["Mg", "O"],
    coords=[[0, 0, 0], [0.5, 0.5, 0.5]],
)

And we finally take this input `Structure` and run it through our newly created workflow.

In [ ]:
dispatch_id = ct.dispatch(workflow)(structure)
result = ct.get_result(dispatch_id, wait=True)
print(result)